In [2]:
import pandas as pd

users = [
    {"user_id": 1, "signup_time": "2026-01-25 10:00", "country": "PL"},
    {"user_id": 2, "signup_time": "2026-01-25 12:30", "country": "DE"},
    {"user_id": 3, "signup_time": "2026-01-26 09:10", "country": "PL"},
]

events = [
    {"user_id": 1, "event_time": "2026-01-25 10:05", "event": "visit"},
    {"user_id": 1, "event_time": "2026-01-25 10:30", "event": "purchase"},
    {"user_id": 2, "event_time": "2026-01-25 12:20", "event": "visit"},  # before signup
    {"user_id": 2, "event_time": "2026-01-25 12:40", "event": "visit"},
    {"user_id": 4, "event_time": "2026-01-26 11:00", "event": "visit"},  # user not found
]

df_users = pd.DataFrame(users)
df_events = pd.DataFrame(events)

# keep all events, user data may be missing
df = df_events.merge(df_users, on="user_id", how="left")

df["event_time"] = pd.to_datetime(df["event_time"])
df["signup_time"] = pd.to_datetime(df["signup_time"])

# valid event = user exists AND event after signup
df_valid = df[
    df["signup_time"].notna() &
    (df["event_time"] >= df["signup_time"])
]

result = (
    df_valid
        .groupby("country")
        .agg(
            valid_events_count=("event", "count"),
            purchase_events_count=("event", lambda x: (x == "purchase").sum())
        )
        .reset_index()
)

print(result)


  country  valid_events_count  purchase_events_count
0      DE                   1                      0
1      PL                   2                      1
